<a href="https://colab.research.google.com/github/danan987/PythonSeeding/blob/main/SeedTransactionDetails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Iterating through pandas dataframes
- We can use the DataFrame method iterrows() to create a list of the DataFrame rows that we access through variables

In [ ]:
import pandas as pd

In [ ]:
google_drive_url = "https://docs.google.com/spreadsheets/d/10QIbM1dUJpi6RRPMmQb2F_URgg-dHkFi/edit?usp=sharing&ouid=117218510125754224635&rtpof=true&sd=true"
raw_url = 'https://drive.google.com/uc?id=' + google_drive_url.split('/')[-2]

In [ ]:
sheet_names = ["Transaction Details","Schedule","Transactions"]

In [ ]:
data = pd.read_excel(raw_url, sheet_name = sheet_names)

In [ ]:
data["Transaction Details"]
schedule_mapping = {}

for schedule in data["Schedule"].to_dict(orient="records"):
    # Extracting schedule information
    schedule_id = schedule["ScheduleID"]
    schedule_movie = schedule["Movie"]
    schedule_date_time = schedule["DateTime"]
    schedule_theater = schedule["Theater"]

    # Store the schedule information in the mapping
    schedule_mapping[(schedule_date_time, schedule_movie)] = schedule_id


In [ ]:
transaction_details_string = ""
purchase_id = -1
transaction_id = 0
schedulenum = 0
for i, row in enumerate(data["Transaction Details"].to_dict(orient="records")):
  schedulenum +=1

  if row["Purchase ID"] > purchase_id:
    purchase_id = row["Purchase ID"]
    transaction_id = 0
  transaction_id += 1
  seniorDiscount = "false"
  if(row["Senior Discount?"] == "Y"):
    seniorDiscount = "true"
  year = row["Date/Time"].year
  month = row["Date/Time"].month
  day = row["Date/Time"].day
  hour = row["Date/Time"].hour
  schedule_movie = row["Movie"]
  minute = row["Date/Time"].minute
  second = row["Date/Time"].second
  schedule_date_time = row["Date/Time"]
  schedule_theater = row["Theater"]
  if schedule_theater =="2":
    schedule_theater ="Theater 2"
  else:
    schedule_theater ="Theater 1"


    # Retrieve ScheduleID from the mapping
  schedule_id = schedule_mapping.get((schedule_date_time,schedule_movie))

  payment_method = 'CardCash'

  if row['Payment Method'] == 'Points':
    payment_method = 'PopcornPoint'

  transaction_details_string += f"""
               var scheduleSlot{schedulenum} = context.ScheduleSlots.Include(m=>m.TicketPrice).SingleOrDefault( m => m.ScheduleID == {schedule_id});

            AllTransactionDetails.Add( new TransactionDetail()
            {{

                Transaction = context.Transactions.SingleOrDefault( m => m.TransactionID == {transaction_id}),
                ScheduleSlot = context.ScheduleSlots.SingleOrDefault( m => m.ScheduleID == {schedule_id}),
                SeatLocation = SeatLocation.{row["Seat"]},
                MoviePrice = scheduleSlot{schedulenum}.TicketPrice.Price.Value,
                ExtendedPrice = scheduleSlot{schedulenum}.TicketPrice.Price.Value,

        }});

"""


In [ ]:
print(transaction_details_string)


               var scheduleSlot1 = context.ScheduleSlots.Include(m=>m.TicketPrice).SingleOrDefault( m => m.ScheduleID == 8);

            AllTransactionDetails.Add( new TransactionDetail()
            {
              
                Transaction = context.Transactions.SingleOrDefault( m => m.TransactionID == 1),
                ScheduleSlot = context.ScheduleSlots.SingleOrDefault( m => m.ScheduleID == 8),
                SeatLocation = SeatLocation.A1,
                MoviePrice = scheduleSlot1.TicketPrice.Price.Value,
                ExtendedPrice = scheduleSlot1.TicketPrice.Price.Value,

        });


               var scheduleSlot2 = context.ScheduleSlots.Include(m=>m.TicketPrice).SingleOrDefault( m => m.ScheduleID == 8);

            AllTransactionDetails.Add( new TransactionDetail()
            {
              
                Transaction = context.Transactions.SingleOrDefault( m => m.TransactionID == 2),
                ScheduleSlot = context.ScheduleSlots.SingleOrDefault( m =>

In [ ]:
file = open("SeedTransactionDetails", "w")
file.write(transaction_details_string)
file.close()